<a href="https://colab.research.google.com/github/TonyQ-Lab/supervised-learning/blob/main/diabetics-prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading data

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kandij/diabetes-dataset")

print("Path to dataset files:", path)

100%|██████████| 8.92k/8.92k [00:00<00:00, 12.5MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/kandij/diabetes-dataset/versions/1
